In [41]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

In [42]:
data = pd.read_csv('경상대_날씨\경상대_일자별관측.csv')

# 전처리 column
# WD, WS, PA, PS, TA, TD, HM, PV, RN, RN.1, RN.2, CA, CA.1, CH, VS, SS, SI, TS

In [43]:
data.shape

(26255, 46)

In [44]:
def preprocess_weather_data(df, missing_values=[-9, -99]):
    """
    날씨 데이터 전처리 및 보간 함수
    
    Parameters:
    df: 원본 데이터프레임
    missing_values: 결측값으로 처리할 값들의 리스트
    """
    # 데이터 복사
    df_cleaned = df.copy()
    
    # 결측값 처리
    df_cleaned = df_cleaned.replace(missing_values, np.nan)
    
    # 시계열 인덱스 생성
    df_cleaned['datetime'] = pd.to_datetime(df_cleaned['YYMMDDHHMI'], format='%Y%m%d%H%M')
    df_cleaned.set_index('datetime', inplace=True)
    
    # 수치형 열만 선택 (object 타입 제외)
    numeric_columns = df_cleaned.select_dtypes(include=[np.number]).columns
    
    # 각 열별 결측값 비율 계산
    missing_ratio = df_cleaned[numeric_columns].isnull().sum() / len(df_cleaned) * 100
    
    print("각 열의 결측값 비율 (%):")
    print(missing_ratio.sort_values(ascending=False))
    
    return df_cleaned

In [45]:
def interpolate_weather_data(df, columns_to_interpolate, method='spline'):
    """
    선택된 열에 대해 보간 수행
    
    Parameters:
    df: 전처리된 데이터프레임
    columns_to_interpolate: 보간할 열 리스트
    method: 보간 방법 ('spline', 'linear', 'time')
    """
    df_interpolated = df.copy()
    
    for column in columns_to_interpolate:
        if method == 'spline':
            # 결측치가 아닌 데이터 포인트만 선택
            valid_data = df_interpolated[column].dropna()
            if len(valid_data) > 3:  # 스플라인은 최소 4개의 포인트 필요
                # 숫자형 인덱스 생성
                x = np.arange(len(valid_data))
                # 스플라인 보간
                cs = CubicSpline(x, valid_data.values)
                # 모든 인덱스에 대해 보간값 계산
                all_x = np.arange(len(df_interpolated))
                interpolated_values = cs(all_x)
                df_interpolated[f'{column}_interpolated'] = interpolated_values
        else:
            # 선형 또는 시계열 보간
            df_interpolated[f'{column}_interpolated'] = df_interpolated[column].interpolate(
                method='linear' if method == 'linear' else 'time'
            )
    
    return df_interpolated

In [46]:
def interpolate_missing_values_all_columns(df, missing_values=[-9, -99], spline_order=3, fill_value=0):
    """
    주어진 26255x46 형태의 데이터프레임에서 결측치(-9, -99)를 스플라인 보간으로 채웁니다.
    모든 값이 결측치인 열은 지정된 값으로 채웁니다.
    
    Parameters:
    df (pd.DataFrame): 결측치가 -9 또는 -99로 표시된 26255x46 데이터프레임
    missing_values (list): 결측치를 나타내는 값들의 리스트
    spline_order (int): 스플라인 보간의 차수
    fill_value (numeric): 모든 값이 결측치인 열을 채울 값
    
    Returns:
    pd.DataFrame: 결측치가 보간된 데이터프레임
    """
    # 1. 결측치로 대체
    df_replaced = df.replace(missing_values, np.nan)
    
    # 2. 결측치 비율 계산
    missing_ratio = df_replaced.isnull().mean()
    
    # 3. 모든 값이 결측치인 열 식별 (missing_ratio == 1.0)
    all_missing_cols = missing_ratio[missing_ratio == 1.0].index.tolist()
    
    if all_missing_cols:
        print(f"모든 값이 결측치인 열을 '{fill_value}'으로 채웁니다: {all_missing_cols}")
        # 모든 값이 결측치인 열을 지정된 값으로 채웁니다.
        df_replaced[all_missing_cols] = fill_value
    
    # 4. 스플라인 보간 수행
    df_interpolated = df_replaced.interpolate(method='spline', order=spline_order, axis=0, limit_direction='both')
    
    # 5. 보간 후 여전히 결측치가 있는지 확인
    remaining_missing = df_interpolated.isnull().sum().sum()
    if remaining_missing > 0:
        print(f"보간 후에도 {remaining_missing}개의 결측치가 남아있습니다. 추가로 앞/뒤 채우기를 수행합니다.")
        # 추가적으로 앞/뒤 방향으로 채웁니다.
        df_interpolated = df_interpolated.fillna(method='bfill').fillna(method='ffill')
    
    return df_interpolated

In [47]:
print("변환 전 데이터 샘플:")
print(data.head())

data = data.replace([-9, -99], np.nan)

변환 전 데이터 샘플:
     YYMMDDHHMI  STN  WD   WS  GST  GST.1  GST.2      PA      PS  PT  ...  \
0  202201010000  192   0  0.0   -9   -9.0     -9  1027.3  1031.1   2  ...   
1  202201010100  192  16  0.7   -9   -9.0     -9  1027.7  1031.5  -9  ...   
2  202201010200  192   0  0.1   -9   -9.0     -9  1027.7  1031.5  -9  ...   
3  202201010300  192   0  0.0   -9   -9.0     -9  1027.8  1031.6   2  ...   
4  202201010400  192   7  0.7   -9   -9.0     -9  1027.0  1030.8  -9  ...   

    TS    TE  TE.1  TE.2  TE.3  ST.1   WH  BF  IR  IX  
0 -1.0 -99.0 -99.0 -99.0 -99.0    -9 -9.0  -9   3  -9  
1 -1.2 -99.0 -99.0 -99.0 -99.0    -9 -9.0  -9   3  -9  
2 -1.4 -99.0 -99.0 -99.0 -99.0    -9 -9.0  -9   3  -9  
3 -1.5 -99.0 -99.0 -99.0 -99.0    -9 -9.0  -9   3  -9  
4 -1.9 -99.0 -99.0 -99.0 -99.0    -9 -9.0  -9   3  -9  

[5 rows x 46 columns]


In [48]:
print("data size: ", data.shape)
print("\n처음 몇 줄: ")
print(data.head())

print("\n각 열의 결측값 개수: ")
print(data.isnull().sum())

data size:  (26255, 46)

처음 몇 줄: 
     YYMMDDHHMI  STN    WD   WS  GST  GST.1  GST.2      PA      PS   PT  ...  \
0  202201010000  192   0.0  0.0  NaN    NaN    NaN  1027.3  1031.1  2.0  ...   
1  202201010100  192  16.0  0.7  NaN    NaN    NaN  1027.7  1031.5  NaN  ...   
2  202201010200  192   0.0  0.1  NaN    NaN    NaN  1027.7  1031.5  NaN  ...   
3  202201010300  192   0.0  0.0  NaN    NaN    NaN  1027.8  1031.6  2.0  ...   
4  202201010400  192   7.0  0.7  NaN    NaN    NaN  1027.0  1030.8  NaN  ...   

    TS  TE  TE.1  TE.2  TE.3  ST.1  WH  BF   IR  IX  
0 -1.0 NaN   NaN   NaN   NaN   NaN NaN NaN  3.0 NaN  
1 -1.2 NaN   NaN   NaN   NaN   NaN NaN NaN  3.0 NaN  
2 -1.4 NaN   NaN   NaN   NaN   NaN NaN NaN  3.0 NaN  
3 -1.5 NaN   NaN   NaN   NaN   NaN NaN NaN  3.0 NaN  
4 -1.9 NaN   NaN   NaN   NaN   NaN NaN NaN  3.0 NaN  

[5 rows x 46 columns]

각 열의 결측값 개수: 
YYMMDDHHMI        0
STN               0
WD                5
WS                5
GST           25979
GST.1         25979
GST

In [49]:
cleaned_data = preprocess_weather_data(data)
cleaned_data

각 열의 결측값 비율 (%):
ST.1          100.000000
TE.3          100.000000
TE.2          100.000000
TE.1          100.000000
WH            100.000000
SD.1          100.000000
WC            100.000000
WP            100.000000
CT.1          100.000000
SD.2          100.000000
SD            100.000000
RN.3          100.000000
BF            100.000000
TE            100.000000
CT.3          100.000000
CT.2          100.000000
ST            100.000000
GST            98.948772
GST.2          98.948772
GST.1          98.948772
IX             97.531899
RN             91.811084
RN.1           77.055799
RN.2           77.055799
PR             66.684441
PT             66.680632
SI             45.271377
SS             45.267568
CH             41.653018
VS              1.363550
CA.1            0.990288
TD              0.312321
CA              0.175205
PV              0.099029
HM              0.099029
TS              0.030470
TA              0.026662
WD              0.019044
WS              0.019044
IR      

,YYMMDDHHMI,STN,WD,WS,GST,GST.1,GST.2,PA,PS,PT,...,TS,TE,TE.1,TE.2,TE.3,ST.1,WH,BF,IR,IX
datetime,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,202201010000,192,0.0,0.0,NaN,NaN,NaN,1027.3,1031.1,2.0,...,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
2022-01-01 01:00:00,202201010100,192,16.0,0.7,NaN,NaN,NaN,1027.7,1031.5,NaN,...,-1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
2022-01-01 02:00:00,202201010200,192,0.0,0.1,NaN,NaN,NaN,1027.7,1031.5,NaN,...,-1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
2022-01-01 03:00:00,202201010300,192,0.0,0.0,NaN,NaN,NaN,1027.8,1031.6,2.0,...,-1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
2022-01-01 04:00:00,202201010400,192,7.0,0.7,NaN,NaN,NaN,1027.0,1030.8,NaN,...,-1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,202412311900,192,0.0,0.2,NaN,NaN,NaN,1017.1,1020.8,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0
2024-12-31 20:00:00,202412312000,192,0.0,0.0,NaN,NaN,NaN,1017.9,1021.6,NaN,...,-0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0
2024-12-31 21:00:00,202412312100,192,0.0,0.0,NaN,NaN,NaN,1018.5,1022.2,2.0,...,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0


In [52]:

# 보간 함수 적용
df_filled = interpolate_missing_values_all_columns(data, fill_value=0)

# 결과 확인
print(df_filled.head())

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [51]:
remaining_missing = df_filled.isnull().sum().sum()
print(f"남아있는 결측치의 총 개수: {remaining_missing}")

남아있는 결측치의 총 개수: 0
